# `메타데이터` 추출

## LLM 기반 자동 메타데이터 추출

문서에서 `비즈니스적으로 중요한 메타데이터`를 LLM을 사용하여 자동으로 추출합니다.

### 추출 가능한 메타데이터

**문서 기본 정보:**
- 문서 유형 (보고서, 계약서, 매뉴얼 등)
- 작성 날짜, 수정 날짜
- 저자, 부서, 회사명

**비즈니스 정보:**
- 주요 주제/토픽
- 핵심 키워드
- 관련 프로젝트/제품명
- 대상 고객/시장

### 왜 메타데이터가 중요한가?

1. **검색 필터링**: 특정 날짜/부서/프로젝트 문서만 검색
2. **권한 관리**: 부서별 접근 제어
3. **분석**: 문서 트렌드, 주제 분포 파악
4. **비즈니스 로직**: 업무 규칙 적용

### **주의사항**

**잘못된 메타데이터의 위험:**
- 검색 결과 오염
- 의미 검색 방해 (불필요한 키워드)
- 사용자 혼란

**베스트 프랙티스:**
- 표준화된 스키마 사용
- LLM 검증 (confidence score)
- 인간 검토 (Human-in-the-loop)
- 최소한의 필수 메타데이터만, 특히 엔지니어링적인 메타데이터는 지양

## 비즈니스 메타데이터 활용 및 Qdrant 필터링

### 왜 비즈니스 메타데이터가 중요한가?

기술적 메타데이터 (page, chunk_id)와 달리, **비즈니스 메타데이터**는 실제 사용자 질의에 대한 검색 정확도를 높입니다:

- **날짜/기간**: "2023년 실적" → 필터링으로 정확한 문서만 검색
- **회사명/부서**: "삼성전자 보고서" → 특정 회사 문서만 검색
- **보고서 타입**: "재무제표", "분기보고서" → 문서 종류로 필터링
- **카테고리**: "금융", "법률", "기술" → 도메인 특화 검색

**프로덕션 안티패턴**:
- 과도한 메타데이터 (chunk_size, embedding_model 등 기술 정보)
- 엔지니어에 의한 자의적인 태그 (중요도, 난이도 등 주관적 평가)
- 검색에 사용하지 않는 메타데이터

In [1]:
# ----------------------------------------------------------------------------
# OpenAI / OpenRouter 모델 초기화 헬퍼
# ----------------------------------------------------------------------------
import os
from typing import Literal

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()


def _resolve_api_context() -> tuple[str, str]:
    """선택된 API 키와 베이스 URL 정보를 반환합니다."""
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        raise RuntimeError("OPENROUTER_API_KEY가 필요합니다.")

    base_url = os.getenv("OPENROUTER_API_BASE") or "https://openrouter.ai/api/v1"

    return (api_key, base_url)


def create_openrouter_llm(
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.3,
    max_tokens: int | None = None,
    **kwargs: object,
) -> ChatOpenAI:
    """OpenAI 호환 LLM 생성 헬퍼.

    Args:
        model: 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/gpt-4o, anthropic/claude-3-sonnet, google/gemini-pro)
        temperature: 생성 온도 (0.0-2.0)
        max_tokens: 최대 생성 토큰 수

    Returns:
        ChatOpenAI: 설정된 LLM 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    openai_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "temperature": temperature,
        "max_retries": 3,
        "timeout": 60,
        **kwargs,
    }
    if max_tokens is not None:
        openai_kwargs["max_tokens"] = max_tokens
    if base_url:
        openai_kwargs["base_url"] = base_url
    return ChatOpenAI(**openai_kwargs)


def create_embedding_model(
    model: str = "openai/text-embedding-3-small",
    **kwargs,
) -> OpenAIEmbeddings:
    """OpenAI 호환 임베딩 모델 생성.

    Args:
        model: 임베딩 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/text-embedding-3-small, openai/text-embedding-3-large)
        **kwargs: 추가 파라미터 (encoding_format 등은 model_kwargs로 전달됨)

    Returns:
        OpenAIEmbeddings: 설정된 임베딩 모델 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    # 전달받은 kwargs에서 model_kwargs로 전달할 파라미터 분리
    # encoding_format, extra_headers 등은 model_kwargs로 전달
    model_kwargs: dict = {}
    embedding_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "show_progress_bar": True,
        "skip_empty": True,
    }

    # 전달받은 kwargs 처리
    for key, value in kwargs.items():
        # OpenRouter API 특정 파라미터는 model_kwargs로 전달
        if key in ("encoding_format"):
            model_kwargs[key] = value
        else:
            # 나머지는 OpenAIEmbeddings에 직접 전달
            embedding_kwargs[key] = value

    if base_url:
        embedding_kwargs["base_url"] = base_url

    # model_kwargs가 있으면 전달
    if model_kwargs:
        embedding_kwargs["model_kwargs"] = model_kwargs

    return OpenAIEmbeddings(**embedding_kwargs)


def create_embedding_model_direct(
    model: str = "qwen/qwen3-embedding-0.6b",
    encoding_format: Literal["float", "base64"] = "float",
    input_text: str | list[str] = "",
    **kwargs,
) -> list[float] | list[list[float]]:
    """OpenAI SDK를 직접 사용하여 임베딩 생성 (encoding_format 지원).

    LangChain의 OpenAIEmbeddings가 encoding_format을 지원하지 않을 때 사용.

    Args:
        model: 임베딩 모델 이름
        encoding_format: 인코딩 형식 ("float")
        input_text: 임베딩할 텍스트 (문자열 또는 문자열 리스트)
        **kwargs: 추가 파라미터

    Returns:
        임베딩 벡터 리스트 (단일 텍스트) 또는 리스트의 리스트 (여러 텍스트)
    """
    from openai import OpenAI

    api_key, base_url = _resolve_api_context()

    client = OpenAI(
        base_url=base_url,
        api_key=api_key,
    )

    # input_text가 비어있으면 kwargs에서 가져오기
    if not input_text:
        input_text = kwargs.get("input", "")

    response = client.embeddings.create(
        model=model,
        input=input_text,
        encoding_format=encoding_format,
    )

    # 단일 텍스트인 경우 첫 번째 임베딩 반환
    if isinstance(input_text, str):
        return response.data[0].embedding
    else:
        # 여러 텍스트인 경우 모든 임베딩 반환
        return [item.embedding for item in response.data]


def get_available_model_types() -> dict[str, list[str]]:
    """OpenRouter에서 사용 가능한 모델 유형을 반환합니다.

    Returns:
        dict[str, list[str]]: 모델 유형별 모델 목록
    """
    return {
        "chat": [
            "openai/gpt-4.1",
            "openai/gpt-4.1-mini",
            "openai/gpt-5",
            "openai/gpt-5-mini",
            "anthropic/claude-sonnet-4.5",
            "anthropic/claude-haiku-4.5",
            "google/gemini-2.5-flash-preview-09-2025",
            "google/gemini-pro-2.5",
            "x-ai/grok-4-fast",
            "moonshotai/kimi-k2-thinking",
            "liquid/lfm-2.2-6b",
            "z-ai/glm-4.6",
        ],
        "embedding": [
            "openai/text-embedding-3-small",
            "openai/text-embedding-3-large",
            "google/gemini-embedding-001",
            "qwen/qwen3-embedding-0.6b",
            "qwen/qwen3-embedding-4b",
            "qwen/qwen3-embedding-8b",
        ],
    }


embeddings = create_embedding_model()
llm = create_openrouter_llm()

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

In [9]:
# NOTE: Parsing 완료된 문서를 Load 해와야함
from pathlib import Path

try:
    markdown_text = ""
    chunks_dir = Path("./chunks")
    suffix = "_docling"

    # NOTE: 파일 이름에 따라 파일 읽기
    for md_path in chunks_dir.glob(f"*{suffix}.md"):
        with open(md_path) as f:
            markdown_text = f.read()

    print(f"읽어온 파일 길이 확인: {len(markdown_text)}")

    # 1. Pydantic 모델로 메타데이터 스키마 정의
    class DocumentMetadata(BaseModel):
        """Document metadata."""

        document_type: str = Field(
            description="Document type (e.g., legal, technical, report, contract)"
        )
        main_topic: str = Field(description="Primary topic of the document (one sentence)")
        keywords: list[str] = Field(description="3 to 5 core keywords (nouns, no duplicates)")
        date_mentioned: str | None = Field(
            default=None,
            description=(
                "A single, most relevant date mentioned in the document "
                "(YYYY-MM-DD). Use null if not determinable."
            ),
        )
        entities: list[str] = Field(
            description=("Up to 5 named entities (persons, organizations, laws, products)")
        )
        summary: str = Field(description="Document summary in 2-3 sentences (Korean output)")

    # 3. 프롬프트 템플릿
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are a Korean document metadata extraction agent.
Extract metadata that strictly conforms to the schema and output exact JSON only.

[Goals]
- Detect and leverage document type to apply optimal, field-specific rules.
- The result must be JSON matching the schema below exactly (order free, no extra keys).

[Language]
- All field values must be written in Korean. Proper nouns may keep their original script.

[Document type decision]
- If a document_type_hint is provided, honor it with highest priority.
- Otherwise infer from content. Typical categories:
  - Legal: "법률", "시행령/시행규칙", "제○○호", "공포/시행", "조/항/목"
  - Technical: "알고리즘/모델/버전", "성능/평가", "데이터셋", "아키텍처"
  - Report: "현황/분석/결과/제언", "기간/지표/조사", "요약/목차"
  - Contract: "계약서/당사자/갑/을/유효기간/해지/서명"

[Field rules (common)]
- main_topic: One-sentence statement of the document’s purpose or thesis; keep concise.
- keywords (3–5): Core terms (e.g., policy/technology/clause names). No duplicates or near-synonyms.
- date_mentioned: The single most relevant date in YYYY-MM-DD. If uncertain or format missing parts, use null.
- entities (≤5): Proper nouns (people, organizations, laws, products, parties). Use canonical names. No duplicates.
- summary (2–3 sentences): Purpose → main content → scope/impact, concise.

[Type-specific priorities]
- Legal:
  - date_mentioned priority: Effective date > Promulgation date > Enactment date; if multiple, prefer the most recent effective-related date.
  - keywords: Law/ordinance/rule names, core amendments, key clauses.
  - entities: Law names, authorities (e.g., 고용노동부), ordinance/rule titles.
  - summary: Background → key changes → applicability.
- Technical:
  - date_mentioned: Publication/release/experiment date (most official).
  - keywords: Model/algorithm/framework/dataset/metric.
  - entities: Models/products/datasets/institutions or research groups.
  - summary: Problem → approach/architecture → main results/contributions.
- Report:
  - date_mentioned: Publication date or end of reporting period.
  - keywords: Topic domain, metrics, target, period.
  - entities: Publishing org, project name, subjects/regions.
  - summary: Purpose → methods/data → key findings/recommendations.
- Contract:
  - date_mentioned: Effective date > execution date > signature date (choose a single, definite date).
  - keywords: Contract type, scope/object, term, consideration/payment, termination.
  - entities: Parties (company/person names), brand/product.
  - summary: Parties/purpose → key obligations/term → price/termination.

[Normalization/format]
- Dates must be "YYYY-MM-DD". If month/day are missing or ambiguous, use null.
- Remove noise (e.g., [1], (주), ㈜), extra brackets/footnotes, and duplicates.
- Normalize spacing and punctuation.

[Quality checks]
- Keys, types, and formats must match the schema exactly.
- If a value is empty or uncertain, use null (especially date_mentioned).
- Do NOT output anything other than JSON (no explanations or code fences).

[Output schema]
```json
{{
  "document_type": string,
  "main_topic": string,
  "keywords": string[3..5],
  "date_mentioned": string | null,
  "entities": string[0..5],
  "summary": string
}}
```

[Procedure (internal)]
1) Decide document type (hint first, else infer).
2) Collect candidates, refine via type-specific rules.
3) Validate format/constraints (keys/types/date format/length).
4) Output JSON only.
""",
            ),
            (
                "human",
                """아래 문서를 분석해 스키마에 맞춘 메타데이터를 JSON으로만 출력하세요.

[문서]
<document>
{document_text}
</document>
""",
            ),
        ]
    )

    result: DocumentMetadata = llm.with_structured_output(DocumentMetadata).invoke(
        prompt.format(
            document_text=markdown_text,
        )
    )

    print(result)

    # 6. 결과 출력
    print("추출된 메타데이터:")
    print("-" * 80)
    print(f"문서 유형: {result.document_type}")
    print(f"주요 주제: {result.main_topic}")
    print("\n핵심 키워드:")
    for kw in result.keywords:
        print(f"  - {kw}")
    print(f"\n날짜: {result.date_mentioned or '(없음)'}")
    print("\n주요 개체:")
    for entity in result.entities:
        print(f"  - {entity}")
    print("\n요약:")
    print(f"  {result.summary}")
    print("-" * 80)

except Exception as e:
    print(f"오류: {type(e).__name__}")
    print(f"메시지: {str(e)}")

읽어온 파일 길이 확인: 41431
document_type='법률' main_topic='근로기준법은 근로조건의 기준을 정하여 근로자의 기본적 생활 보장과 국민경제 발전을 목적으로 한다.' keywords=['근로기준법', '근로조건', '해고', '임금', '근로시간'] date_mentioned='2025-02-23' entities=['고용노동부', '노동위원회', '근로기준법', '산업안전보건법', '행정소송법'] summary='이 법률은 근로자의 근로조건 기준을 규정하여 기본적 생활을 보장하고 국민경제 발전을 도모한다. 주요 내용은 근로계약, 임금, 근로시간과 휴식, 여성과 소년 근로자 보호, 안전과 보건, 재해보상, 취업규칙, 기숙사 운영, 근로감독 및 벌칙 규정으로 구성된다. 이 법은 2025년 2월 23일부터 시행되며, 사업장과 근로자 모두에게 적용된다.'
추출된 메타데이터:
--------------------------------------------------------------------------------
문서 유형: 법률
주요 주제: 근로기준법은 근로조건의 기준을 정하여 근로자의 기본적 생활 보장과 국민경제 발전을 목적으로 한다.

핵심 키워드:
  - 근로기준법
  - 근로조건
  - 해고
  - 임금
  - 근로시간

날짜: 2025-02-23

주요 개체:
  - 고용노동부
  - 노동위원회
  - 근로기준법
  - 산업안전보건법
  - 행정소송법

요약:
  이 법률은 근로자의 근로조건 기준을 규정하여 기본적 생활을 보장하고 국민경제 발전을 도모한다. 주요 내용은 근로계약, 임금, 근로시간과 휴식, 여성과 소년 근로자 보호, 안전과 보건, 재해보상, 취업규칙, 기숙사 운영, 근로감독 및 벌칙 규정으로 구성된다. 이 법은 2025년 2월 23일부터 시행되며, 사업장과 근로자 모두에게 적용된다.
-------------------------------------------------------------------------

## 메타데이터 활용 전략 및 주의사항

###  좋은 메타데이터 설계

**1. 최소주의 (Minimalism)**
```python
# 좋은 예: 필수 정보만
metadata = {
    "doc_type": "legal",
    "date": "2025-02-23",
    "source": "labor_law.pdf"
}

# 나쁜 예: 불필요한 정보 과다
metadata = {
    "doc_type": "legal",
    "sub_type": "law",
    "category": "labor",
    "subcategory": "working_hours",
    "tags": ["law", "labor", "korea", "employment", "regulation"],
    # ... 검색에 방해가 됨
}
```

**2. 표준화 (Standardization)**
```python
# 좋은 예: 표준 형식
date_format = "YYYY-MM-DD"  # ISO 8601
doc_types = ["legal", "technical", "financial"]  # 고정된 enum

# 나쁜 예: 비일관적 형식
dates = ["2025-02-23", "23/02/2025", "Feb 23 2025"]  # 혼란
doc_types = ["legal", "Legal", "법률", "law"]  # 중복
```

**3. 검증 (Validation)**
```python
from pydantic import BaseModel, validator

class ValidatedMetadata(BaseModel):
    doc_type: str
    date: str
    
    @validator('doc_type')
    def validate_doc_type(cls, v):
        allowed = ['legal', 'technical', 'financial']
        if v not in allowed:
            raise ValueError(f'doc_type must be one of {allowed}')
        return v
    
    @validator('date')
    def validate_date(cls, v):
        # YYYY-MM-DD 형식 검증
        import re
        if not re.match(r'^\d{4}-\d{2}-\d{2}$', v):
            raise ValueError('date must be YYYY-MM-DD format')
        return v
```

### 안티패턴

**1. 과도한 키워드 기반 태깅**
```python
# 문제: 의미 검색을 방해
bad_metadata = {
    "keywords": [
        "근로", "기준", "법", "노동", "시간", "임금", "휴가", 
        "해고", "계약", "근로자", "사용자", "고용", "퇴직",
        # ... 너무 많음 → 검색 노이즈
    ]
}

# 해결: 핵심 3-5개만
# 여긴 거의 경험적 영역입니다. 5개 미만이 가장 적절했습니다. 
good_metadata = {
    "keywords": ["근로기준법", "근로시간", "임금"]
}
```

**2. 엔지니어의 임의 메타데이터**
> 가장 중요한 부분입니다. 절대로 `엔지니어` 가 임의 판단한 데이터는 들어가지 않는게 좋습니다.
> 왜냐하면, 엔지니어링적으로 필요한 메타데이터는 RDB 의 PK 성격을 가지는 경우가 대부분인데, 그러한 경우 이미 벡터 Collection 구성에 대한 시스템 설계 단에서 이미 처리가 되어야 하는 경우가 대부분.

```python
# 문제: 비즈니스 로직과 무관한 기술 메타데이터
bad_metadata = {
    "parser_version": "1.2.3",
    "chunk_algorithm": "recursive",
    "embedding_model": "text-embedding-3-small",
    # → 검색/필터링에 불필요
}

# 해결: 비즈니스 메타데이터만
good_metadata = {
    "department": "HR",
    "confidential": False,
    "effective_date": "2025-01-01"
}
```

**3. 중복 정보**
```python
# 문제: 문서 내용과 중복
bad_metadata = {
    "title": "근로기준법",  # 이미 문서에 있음
    "first_paragraph": "제1조...",  # 이미 문서에 있음
    # → 임베딩 중복, 검색 노이즈
}

# 해결: 문서에 없는 정보만
good_metadata = {
    "law_number": "제20520호",  # 추가 정보
    "enactment_date": "2025-02-23",  # 추가 정보
}
```

---

### 실무 권장사항

**1. 업무 담당자가 꼭 Human-in-the-Loop 기반의 검증!!**
```python
# LLM 추출 후 인간 검토
extracted_metadata = llm.extract_metadata(document)

# Confidence score 확인
if extracted_metadata.confidence < 0.8:
    # 낮은 신뢰도 → 인간 검토 필요
    send_to_review_queue(document, extracted_metadata)
else:
    # 높은 신뢰도 → 자동 승인
    save_metadata(extracted_metadata)
```

**2. 점진적 개선(한번에 다 못뽑습니다, 절대로!)**
```python
# 초기: 최소 메타데이터로 시작
v1_metadata = {
    "doc_type": "legal",
    "date": "2025-02-23"
}

# 이후: 사용자 피드백 기반으로 확장
if user_feedback.needs_department_filter:
    v2_metadata = {
        **v1_metadata,
        "department": "HR"
    }
```

# Text Data - Context Engineering

주요 목적:   
- 의미 검색 시 잘 걸려 나올 수 있도록 하기 위함(=Contextual)
- 복잡한 텍스트나 도메인 전문 용어의 사용 시 해당 용어를 풀어서 설명하거나 단어 사전을 이용해 매핑하여 치환. 


## 메타데이터 전략

| 유형 | 예시 | 활용 방법 |
|------|------|----------|
| **구조적** | page, section, chunk_type | 필터링, 정렬 |
| **비즈니스** | 날짜, 회사명, 보고서 타입 | 검색 정확도 향상 |
| **의미적** | 요약, 키워드, 카테고리 | 재순위화, 다중 벡터 |


## **핵심 포인트**

- 상위 문맥 추가: 청크에 섹션 제목, 문서 제목 포함
- 자동 메타데이터: LLM으로 추출하여 검색 정확도 향상
- 필터링 최적화: 자주 사용하는 필드는 Qdrant 인덱스 생성

In [ ]:
# TODO: 메타데이터를 Context 에 다시 집어넣어서 Chunk 의 내용을 풍부하게 만들어보면 어떨까요?